# Código para generar el reporte base del dashboard
Consideraciones:
- Reporte 20.06 --> Actividades registradas por el contratista a cargo de gestionar la cobranza en campo.
- Deuda 1 dia antes --> Archivo que contiene a los clientes con 3 a más meses de deuda.
- Deuda 2 meses --> Archivo que contiene a los clientes con 2 meses de deuda.
- Deuda final --> La deuda actual luego de haber realizado la gestión de cobranza.

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from datetime import timedelta
import re
import gzip
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
#El display.max.columns es para poder ver todas las columnas del dataframe

In [ ]:
Tecsur = pd.read_excel('Reporte 20.06.xlsx', header = 1)
coordenadas = pd.read_parquet('datos_clientes_activos_complementado_20240529_20240530094833.gzip')
DeudaFinal = pd.read_excel("Deuda final.xlsx")
dtypes = {9:'object', 10:'object'}
DeudaInicial = pd.read_csv("Deuda 2 meses.csv", dtype=dtypes)
dtypes1 = {13:'object', 74:'object', 75:'object', 78:'object', 80:'object', 81:'object', 84:'object'}
DeudaInicial1 = pd.read_csv("Deuda 1 dia antes.csv", dtype = dtypes1)
Tecsur.drop('Prioridad', axis=1, inplace=True)
#axis=1 --> Elimina columnas
#axis=0 --> Elimina filas
#Si quieres eliminar varias columnas o filas --> CoR_a_eliminar = ['A','B'] O [1,2]
#Los dtype convierten esos índices/columnas en el tipo de dato "object" para evitar el warning
#Para ver que nombre de los índices usar df.columns[x]
lista = Tecsur['Número Suministro']
coordenadas['SUMINISTRO'] = coordenadas['SUMINISTRO'].astype('int64')
coordenadas_filtrado = coordenadas[coordenadas['SUMINISTRO'].isin(lista)]
coordenadas_filtrado = coordenadas_filtrado[['SUMINISTRO','LATITUD X','LONGITUD Y']]
coordenadas_filtrado = coordenadas_filtrado.rename(columns={'SUMINISTRO':'Número Suministro', 'LATITUD X':'COORDENADA DE LATITUD', 'LONGITUD Y':'COORDENADA DE LONGITUD'})
Tecsur_1 = pd.merge(Tecsur, coordenadas_filtrado, on = 'Número Suministro', how = 'left')
indice = Tecsur_1.columns.get_loc('Fecha Ejecución')
Tecsur_1.insert(indice + 1, 'F. Generación', Tecsur_1['Fecha Ejecución'])
#La lista se creó para probar otro método de filtrar y obtener las columnas que se necesitaban
#Fue necesario cambiar el tipo de dato de coordenadas['SUMINISTRO'] para que sea el mismo de Tecsur['Número Suministro'],
#de lo contrario el merge se generaba como vacío y todos los datos resultaban en NaN
#La función insert se utilizó para colocar la columnas F.Generación en el índice requerido
DeudaFinal = DeudaFinal.rename(columns={'CLIENTE':'Número Suministro'})
pattern = r"-?\d+$"
DeudaFinal = DeudaFinal.replace(pattern, "", regex=True)
DeudaFinal['Número Suministro'] = DeudaFinal['Número Suministro'].apply(lambda x: re.sub(r"-[A-Za-z]$", "", x))
#El código de la línea 3 elimina los números que están después del guión de los suministros
#El código de la línea 4 elimina las "k" que están después del guión de los suministros
DeudaFinal['Número Suministro'] = DeudaFinal['Número Suministro'].astype('Int64')
DeudaFinal_filtrado = DeudaFinal[['Número Suministro', 'DEUDA TOTAL', 'ANTIGUEDAD']]
DeudaFinal_filtrado = DeudaFinal_filtrado.rename(columns={'DEUDA TOTAL':'Deuda final', 'ANTIGUEDAD':'Antigüedad final'})
Tecsur_2 = pd.merge(Tecsur_1, DeudaFinal_filtrado, on = 'Número Suministro', how = 'left')
#Se filtra el dataframe ""Deuda final" y se hace un merge
#Hagamos un drop de las columnas NaN y luego un concat de Tecsur_3 y Tecsur_4
lista1 = Tecsur_2['Número Suministro']
DeudaInicial_filtrado = DeudaInicial[DeudaInicial['SUMINISTRO'].isin(lista1)]
DeudaInicial_filtrado = DeudaInicial_filtrado[['SUMINISTRO','SALDO','ANTIGÜEDAD DEUDA']]
DeudaInicial_filtrado = DeudaInicial_filtrado.rename(columns={'SUMINISTRO':'Número Suministro', 'SALDO':'Deuda Inicial', 'ANTIGÜEDAD DEUDA':'Antigüedad inicial'})
Tecsur_3 = pd.merge(Tecsur_2, DeudaInicial_filtrado, on = 'Número Suministro', how = 'left')
Tecsur_3.dropna(subset=['Deuda Inicial'], inplace=True)
#Se aplica el mismo método para el friltrado de columnas y el merge
#Se utiliza el dropna para eliminar las filas que que contienen "NaN" en la columna Deuda Inicial
DeudaInicial1_filtrado = DeudaInicial1[DeudaInicial1['CLIENTE'].isin(lista1)]
DeudaInicial1_filtrado = DeudaInicial1_filtrado[['CLIENTE', 'DEUDA TOTAL', 'ANTIGUEDAD']]
DeudaInicial1_filtrado = DeudaInicial1_filtrado.rename(columns={'CLIENTE':'Número Suministro', 'DEUDA TOTAL':'Deuda Inicial', 'ANTIGUEDAD':'Antigüedad inicial'})
Tecsur_4 = pd.merge(Tecsur_2, DeudaInicial1_filtrado, on = 'Número Suministro', how = 'left')
Tecsur_4.dropna(subset=['Deuda Inicial'], inplace=True)
#Con el dropna se están eliminando las filas que contienen "NaN" en la columna Deuda Inicial
Tecsur_final = pd.concat([Tecsur_3, Tecsur_4], ignore_index=True)
Tecsur_final['Recaudación'] = Tecsur_final['Deuda Inicial'] - Tecsur_final['Deuda final']
Tecsur_final['Estado'] = ['Salió de Morosidad' if x < 2 else 'Permanece en Morosidad' for x in Tecsur_final['Antigüedad final']]
#El código de la línea de arriba es lo equivalente de aplicar una función SI en Excel
indice2 = Tecsur_final.columns.get_loc('COORDENADA DE LONGITUD')
Tecsur_final.insert(indice2 + 1, 'Google Maps', "")
indice3 = Tecsur_final.columns.get_loc('Google Maps')
Tecsur_final.insert(indice3 + 1, 'Prioridad', "")
#La función insert (posición index, nombre de columna a insertar, valores de la nueva columna)
new_order = ['Ítem', 'Sucursal', 'Número Suministro', 'Número Medidor',
       'Ruta / Libro', 'Dirección', 'Operador', 'Fecha Solicitud',
       'Fecha Generación', 'Fecha Ejecución', 'F. Generación', 'Fecha Pago',
       'Acción Realizada', 'Situación Encontrada', 'Lectura', 'F - T - A',
       'Sticker', 'Observaciones', 'Tipo Proceso', 'Efectividad',
       'Estado Suministro', 'Cant. Fotos', 'Ubicación Medidor',
       'COORDENADA DE LATITUD', 'COORDENADA DE LONGITUD', 'Google Maps',
       'Prioridad','Deuda Inicial', 'Antigüedad inicial', 'Recaudación', 'Deuda final', 'Antigüedad final', 'Estado']
Tecsur_final = Tecsur_final.reindex(columns = new_order)
Tecsur_final['F. Generación'] = Tecsur_final['F. Generación'].str.split(' ').str[0]
#Con el reindex le asignamos el orden que queremos a las columnas y con str.split eliminamos cualquier valor que esté después del espacio
Tecsur_final.to_excel("Reporte Tecsur 2.xlsx", index=False)